In [4]:
import os
from dotenv import load_dotenv
import sys 


load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")


if not api_key:
    print("🚨 Error: GOOGLE_API_KEY not found in .env file.")
    sys.exit(1) 

if len(api_key) < 30: 
    print("🚨 Error: The provided GOOGLE_API_KEY seems too short to be valid.")
    sys.exit(1)

print("✅ Google API Key loaded and validated successfully.")

✅ Google API Key loaded and validated successfully.


In [ ]:
import os
import json
from datetime import datetime, timedelta
from typing import TypedDict, Annotated, List, Dict, Literal
import copy
from langchain_core.messages import messages_to_dict
from dateutil.parser import parse as parse_date 


from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, BaseMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode


if 'GOOGLE_API_KEY' not in os.environ:
    print("Please set your GOOGLE_API_KEY environment variable.")
    os.environ["GOOGLE_API_KEY"] = "dummy_key_for_testing_setup"


class Topic(TypedDict): name: str; completed: bool
class ScheduleRule(TypedDict): day_of_week: str; time: str
class AttendanceLog(TypedDict): date: str; status: Literal["attended", "missed", "cancelled"]
class SubjectDetails(TypedDict): topics: List[Topic]; schedule_rules: List[ScheduleRule]; attendance_log: List[AttendanceLog]
class Event(TypedDict): type: Literal["quiz", "holiday", "other"]; subject: str | None; date: str; time: str | None; description: str; syllabus: List[str] | None
class GraphState(TypedDict): messages: Annotated[List[BaseMessage], lambda x, y: x + y]; subjects: Dict[str, SubjectDetails]; events: List[Event]


@tool
def set_recurring_class_schedule(subject: str, day_of_week: str, time: str) -> str:
    """Sets a recurring class schedule for a subject on a specific day and time."""
    return f"Rule created to schedule {subject} for every {day_of_week} at {time}."
@tool
def schedule_one_off_event(description: str, date: str, event_type: Literal["quiz", "holiday", "other"], time: str = None, subject: str = None) -> str:
    """Schedules a single, non-recurring event like a quiz, holiday, or special appointment."""
    return f"Scheduled event '{description}' on {date}."
@tool
def link_topics_to_quiz(quiz_description: str, topics: List[str]) -> str:
    """Links a list of topic names to a scheduled quiz to define its syllabus."""
    return f"Syllabus with {len(topics)} topics linked to quiz '{quiz_description}'."
@tool
def log_attendance(subject: str, date: str, status: Literal["attended", "missed"]) -> str:
    """Logs the attendance status for a specific subject on a given date."""
    return f"Attendance for {subject} on {date} logged as {status}."
@tool
def cancel_class_on_date(subject: str, date: str, reason: str) -> str:
    """Cancels a class for a specific subject on a given date, for example, due to a holiday."""
    return f"Class for {subject} on {date} cancelled. Reason: {reason}."
@tool
def view_schedule_for_period(start_date: str, end_date: str) -> str:
    """Retrieves the schedule for a given period, including recurring classes and special events."""
    return f"Schedule data from {start_date} to {end_date} has been retrieved."
@tool
def get_attendance_summary(subject: str) -> str:
    """Calculates and retrieves the attendance summary for a specific subject."""
    return f"Attendance summary for {subject} has been calculated."
@tool
def get_quiz_study_status(quiz_description: str) -> str:
    """Checks the completion status of topics required for a specific quiz."""
    return f"Study status for quiz '{quiz_description}' has been retrieved."

all_tools = [ set_recurring_class_schedule, schedule_one_off_event, link_topics_to_quiz, log_attendance, cancel_class_on_date, view_schedule_for_period, get_attendance_summary, get_quiz_study_status ]


DAY_TO_WEEKDAY = { "monday": 0, "tuesday": 1, "wednesday": 2, "thursday": 3, "friday": 4, "saturday": 5, "sunday": 6 }

def update_state(state: GraphState) -> GraphState:
    print("---UPDATING STATE---")
    last_message = state['messages'][-1]
    if not hasattr(last_message, 'tool_calls') or not last_message.tool_calls: return {}
    new_subjects, new_events = state.get('subjects', {}).copy(), state.get('events', []).copy()
    for tool_call in last_message.tool_calls:
        tool_name, args = tool_call.get('name'), tool_call.get('args', {})
        if tool_name == 'set_recurring_class_schedule':
            subject = args.get('subject')
            if subject not in new_subjects: new_subjects[subject] = {"topics": [], "schedule_rules": [], "attendance_log": []}
            rule = {"day_of_week": args.get('day_of_week'), "time": args.get('time')}
            new_subjects[subject]['schedule_rules'].append(rule)
        elif tool_name == 'schedule_one_off_event':
            event = { "type": args.get('event_type'), "subject": args.get('subject'), "date": args.get('date'), "time": args.get('time'), "description": args.get('description'), "syllabus": [] }
            new_events.append(event)
        elif tool_name == 'link_topics_to_quiz':
            for event in new_events:
                if event.get('type') == 'quiz' and event.get('description') == args.get('quiz_description'): event['syllabus'] = args.get('topics', [])
        elif tool_name == 'log_attendance':
            subject = args.get('subject')
            if subject in new_subjects: new_subjects[subject]['attendance_log'].append({"date": args.get('date'), "status": args.get('status')})
        elif tool_name == 'view_schedule_for_period':
            schedule = []
           
            try:
                start, end = parse_date(args.get('start_date')), parse_date(args.get('end_date')) 
            except (ValueError, TypeError):
                
                error_msg = f"Invalid date format provided. Could not parse dates: {args.get('start_date')}, {args.get('end_date')}"
                tool_message = ToolMessage(content=json.dumps({"error": error_msg}), tool_call_id=tool_call.get('id'))
                state['messages'].append(tool_message)
                continue 
            

            for subject, details in new_subjects.items():
                for rule in details['schedule_rules']:
                    day_num = DAY_TO_WEEKDAY.get(rule['day_of_week'].lower())
                    if day_num is None: continue
                    current = start
                    while current <= end:
                        if current.weekday() == day_num: schedule.append(f"{current.strftime('%Y-%m-%d')} - {subject} Class at {rule['time']}")
                        current += timedelta(days=1)
            for event in new_events:
                event_date = parse_date(event['date']) 
                if start <= event_date <= end: schedule.append(f"{event['date']} - {event['type'].upper()}: {event['description']}")
            schedule.sort()
            tool_message = ToolMessage(content=json.dumps(schedule), tool_call_id=tool_call.get('id'))
            state['messages'].append(tool_message)
    return {"subjects": new_subjects, "events": new_events}

def call_model(state: GraphState):
    print("---AGENT---")
    context = f"Current State:\nSubjects: {json.dumps(state.get('subjects'))}\nEvents: {json.dumps(state.get('events'))}"
    messages_with_context = state['messages'] + [HumanMessage(content=context)]
    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
    model_with_tools = model.bind_tools(all_tools)
    response = model_with_tools.invoke(messages_with_context)
    return {"messages": [response]}

tool_node = ToolNode(all_tools)

def should_continue(state: GraphState):
    last_message = state['messages'][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls: return "continue_to_tools"
    else: return "end_conversation"


workflow = StateGraph(GraphState)
workflow.add_node("agent", call_model); workflow.add_node("update_state", update_state); workflow.add_node("tools", tool_node)
workflow.set_entry_point("agent")
workflow.add_conditional_edges("agent", should_continue, {"continue_to_tools": "update_state", "end_conversation": END})
workflow.add_edge("update_state", "tools"); workflow.add_edge("tools", "agent")
app = workflow.compile()
print("🚀 Academic Dashboard compiled and ready!")

🚀 Academic Dashboard compiled and ready!


In [14]:
def print_state(state):
    state_copy = copy.deepcopy(state)
    state_copy["messages"] = messages_to_dict(state_copy["messages"])
    print(json.dumps(state_copy, indent=2))




current_state = {
    "messages": [],
    "subjects": {},
    "events": []
}


print("--- STEP 1: Setting Schedule ---")
user_input_1 = "Hi! Please set up my schedule. Physics is every Monday at 10:00, and Chemistry is every Thursday at 14:00."
current_state['messages'] = [HumanMessage(content=user_input_1)]
current_state = app.invoke(current_state)

print("\n--- Agent Response 1 ---")
print(current_state['messages'][-1].content)
print("\n--- Current State 1 ---")
print_state(current_state)  



print("\n\n--- STEP 2: Scheduling a Quiz ---")
user_input_2 = f"Great. Now, can you schedule my 'Physics Midterm' quiz for September 22, 2025?"
current_state['messages'].append(HumanMessage(content=user_input_2))
current_state = app.invoke(current_state)

print("\n--- Agent Response 2 ---")
print(current_state['messages'][-1].content)
print("\n--- Current State 2 ---")
print_state(current_state)  



print("\n\n--- STEP 3: Viewing Schedule ---")
user_input_3 = "Perfect. What does my schedule look like from September 15, 2025 to September 21, 2025?"
current_state['messages'].append(HumanMessage(content=user_input_3))
current_state = app.invoke(current_state)

print("\n--- Agent Response 3 ---")
print(current_state['messages'][-1].content)

--- STEP 1: Setting Schedule ---
---AGENT---
---UPDATING STATE---
---AGENT---

--- Agent Response 1 ---
OK. I have added Physics every Monday at 10:00 and Chemistry every Thursday at 14:00 to your schedule. Anything else?

--- Current State 1 ---
{
  "messages": [
    {
      "type": "human",
      "data": {
        "content": "Hi! Please set up my schedule. Physics is every Monday at 10:00, and Chemistry is every Thursday at 14:00.",
        "additional_kwargs": {},
        "response_metadata": {},
        "type": "human",
        "name": null,
        "id": null,
        "example": false
      }
    },
    {
      "type": "ai",
      "data": {
        "content": "",
        "additional_kwargs": {
          "function_call": {
            "name": "set_recurring_class_schedule",
            "arguments": "{\"time\": \"14:00\", \"day_of_week\": \"Thursday\", \"subject\": \"Chemistry\"}"
          }
        },
        "response_metadata": {
          "prompt_feedback": {
            "block

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 44
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing de

---UPDATING STATE---
---AGENT---


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 26
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing de

KeyboardInterrupt: 